```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec


import os
import pyopencl
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import CTCS, CDKLM16, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *
from SWESimulators import DataAssimilationUtils as dautils

from SWESimulators import WindForcingEnsemble

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

# The Lorenz Attractor (Lorenz-63)

Rules of thumb:
- observations every 8th timestep: quasi-linear problem
- observations every 25th timestep: full non-linear problem

In [ ]:

class Lorenz63Ensemble:
        
    def __init__(self, cl_ctx, numParticles, \
                 observation_variance=np.sqrt(2), model_error=np.sqrt(2*0.01), 
                 init_error=np.sqrt(2), obs_vars='111', resample_error=0.0, \
                 T=10):

        self.T = T
        
        self.numParticles = numParticles
        self.particles = [None]*(self.numParticles + 1)
        
        self.obs_index = self.numParticles
                
        self.observation_variance = observation_variance
        self.model_error = model_error
        self.init_error = init_error
        self.resample_error = resample_error
                
        # Use parameters from the PJvL-2010 paper
        self.dt = 0.01
        self.sigma = 10
        self.rho = 28
        self.beta = 8.0/3.0
        self.initValues = [1.508870, -1.531271, 25.46091]
        
        self.covariance_matrix = np.matrix([[1,0.5,0.25], [0.5,1,0.5], [0.25,0.5,1]])
        
        self.timestep = 0
        self.t = 0.0
        
        self.init()
        self.observations = []
        self.obs_vars = obs_vars
        self.obs_matrix = self._create_observation_matrix()
        self.observation_size = str.count(obs_vars, '1')

        
    def _create_observation_matrix(self):
        obs_matrix_array = []
        if self.obs_vars[0]=='1':
            obs_matrix_array.append([1, 0, 0])
        if self.obs_vars[1]=='1':
            obs_matrix_array.append([0,1,0])
        if self.obs_vars[2]=='1':
            obs_matrix_array.append([0,0,1])
        return  np.matrix(obs_matrix_array)
    
    def _observe(self, fullState):
        obs = []
        for i in range(len(fullState)):
            if self.obs_vars[i]=='1':
                obs.append(fullState[i])
        return np.array(obs)

    # ---------------------------------------
    # Override functions we don't need.
    # ---------------------------------------       
    def cleanUp(self):
        pass
        
    def setGridInfo(self, nx, ny, dx, dy, dt, boundaryConditions, eta=None, hu=None, hv=None, H=None):
        pass
    
    def setParameters(self, f=0, g=9.81, beta=0, r=0, wind=None):
        pass
    
    
    # ---------------------------------------
    # Needs to be abstract!
    # ---------------------------------------
    def init(self):
        self.max_timesteps = self.T+1
        for p in range(self.numParticles+1):
            self.particles[p] = np.zeros((3, self.max_timesteps))
            self.particles[p][:,0] = np.random.multivariate_normal(self.initValues, self.init_error**2*self.covariance_matrix )
    
    
    def setParticles(self, newStates):
        for p in range(self.numParticles):
            self.particles[p][:, int(self.timestep)] = newStates[:,p]
            
    
    def observeParticles(self, all=False):
        positions=None
        if all:
            positions = np.zeros((3, self.numParticles))
            for p in range(self.numParticles):
                positions[:,p] = self.particles[p][:,int(self.timestep)]
        else:
            positions = np.zeros((self.observation_size, self.numParticles))
            for p in range(self.numParticles):
                positions[:,p] = self._observe(self.particles[p][:,int(self.timestep)])
        return positions
        
    def observeTrueState(self):
        if len(self.observations) > 0:
            if self.observations[-1][1] == self.timestep:
                return self._observe(self.observations[-1][0])
        
        obs = np.random.multivariate_normal(self.particles[self.obs_index][:,int(self.timestep)], self.observation_variance**2*self.covariance_matrix)
        self.observations.append((obs,self.timestep))
        return self._observe(obs)
    
    
    def _lorenz63_(self, x0):
        x = self.sigma*(x0[1] - x0[0])
        y = x0[0]*(self.rho - x0[2]) - x0[1]
        z = x0[0]*x0[1] - self.beta*x0[2]
        return np.array([x, y, z])
    
    def step(self, t):
        
        k_end = min(self.timestep + t, self.max_timesteps-1)
        for p in self.particles:
            for k in range(int(self.timestep), int(k_end)):
                model_error = np.random.multivariate_normal(np.zeros(3), self.covariance_matrix*self.model_error**2)
                
                # Forward Euler:
                #p[:, k+1] = p[:,k] + self.dt*self._lorenz63_(p[:,k]) + model_error
                
                # RK4:
                k1 = self._lorenz63_(p[:,k])
                k2 = self._lorenz63_(p[:,k] + k1*self.dt/2.0)
                k3 = self._lorenz63_(p[:,k] + k2*self.dt/2.0)
                k4 = self._lorenz63_(p[:,k] + k3*self.dt)
                p[:,k+1] = p[:,k] + self.dt*(k1 + 2*k2 + 2*k3 + k4)/6.0 + model_error
                
        self.timestep = k_end
        self.t = k_end*self.dt
                
    def getDistances(self, obs=None):
        
        pos = self.observeParticles()
        obs = self.observeTrueState()
        dist = np.zeros(self.numParticles)
        for i in range(self.numParticles):
            dist[i] = np.sqrt(np.linalg.norm(pos[:,i]-obs))
            #dist[i] = np.sqrt((pos[:,i]-obs[:])**2 + (pos[1,i]-obs[1])**2 + (pos[2,i]-obs[2])**2  )
        return dist
    
       
    def resample(self, newSampleIndices, reinitialization_variable_notused):
        oldPositions = self.observeParticles(all=True).copy()

        # Make sure to make a clean copy of first resampled particle, and add a disturbance of the next ones.
        resampledOnce = np.full(self.getNumParticles(), False, dtype=bool)
        var = np.eye(3)*self.resample_error
        for i in range(self.numParticles):
            index = newSampleIndices[i]
            if resampledOnce[index]:
                self.particles[i][:,self.timestep] = np.random.multivariate_normal(oldPositions[:, index], var)
            else:
                self.particles[i][:,self.timestep] = oldPositions[:, index]
                resampledOnce[index] = True

        
    def getEnsembleMean(self):
        mean = np.zeros(3)
        for j in range(self.numParticles):
            mean += self.particles[j][:,self.timestep]
        mean = mean/self.numParticles
        return mean
    
        
    def getFullMean(self):
        mean = np.zeros_like(self.particles[0])
        for j in range(self.numParticles):
            mean += self.particles[j]
        mean = mean/self.numParticles
        return mean
    
    def getEnsembleCov(self):
        H = ensemble.getObsMatrix()
        mean = ensemble.getEnsembleMean()

        states = ensemble.observeParticles(True)
        observedStates = np.dot(H, states)

        P_forecast = np.zeros((3,3))
        for i in range(states.shape[1]):
            P_forecast += np.outer(states[:,i]-mean, states[:,i]-mean)
        P_forecast = P_forecast/(ensemble.getNumParticles() - 1.0)
        return P_forecast
    
    def _plot_index(self, i, mean, plot_all):
        particle_number = 0
        
        t = range(0, self.max_timesteps)
        plot_num = self.getNumParticles()
        if not plot_all and plot_num > 100:
            plot_num = 100
            if i == 0:
                print "Plotting only 100 members"
        
        for p in range(plot_num):
            plt.plot(t, self.particles[p][i,:], color="0.8" )
        plt.plot(t, self.particles[self.obs_index][i,:], 'g')
        plt.plot(t, mean[i, :], 'r' )
        if self.obs_vars[i] == '1':
            for o in range(len(self.observations)):
                plt.plot(ensemble.observations[o][1], ensemble.observations[o][0][i], 'bo')
        plt.ylim([-28, 45])
    
    def plotDistanceInfo(self, title=None, obs=None, plot_all=False):
        mean = self.getFullMean()
        fig = plt.figure(figsize=(12,8))
        plt.subplot(3,1,1)    
        self._plot_index(0, mean, plot_all)
        plt.ylabel("$x$")
        plt.xlabel("$t$")
        plt.subplot(3,1,2)    
        self._plot_index(1, mean, plot_all)
        plt.ylabel("$y$")
        plt.xlabel("$t$")
        plt.subplot(3,1,3)    
        self._plot_index(2, mean, plot_all)
        plt.ylabel("$z$")
        plt.xlabel("$t$")
        plt.tight_layout()
        
    
            
    def enforceBoundaryConditions(self):
        self.sim.drifters.enforceBoundaryConditions()
    
    ### Duplication of code
    def getDomainSizeX(self):
        pass
    def getDomainSizeY(self):
        pass
    def getObservationVariance(self):
        return self.observation_variance 
    def getObservationCovMatrix(self):
        if self.obs_vars == '111':
            return self.observation_variance**2 * self.covariance_matrix
        elif self.obs_vars == '110' or self.obs_vars == '011':
            return self.observation_variance**2 * np.matrix([[1.0, 0.5], [0.5, 1.0]])
        elif self.obs_vars == '101':
            return self.observation_variance**2 * np.matrix([[1.0, 0.25], [0.25, 1.0]])
        else:
            return self.observation_variance**2
    def getNumParticles(self):
        return self.numParticles
    def getObsMatrix(self):
        return self.obs_matrix
    


In [ ]:
T = 1000
numParticles = 20
obs_vars = '110'
ensemble = Lorenz63Ensemble(cl_ctx, numParticles, T=T, obs_vars=obs_vars)
                            #obs_matrix_diag=obsmat)
                            #model_error=0, init_error=0.1)
                            #observation_variance=5)

ensemble.step(1000)
ensemble.plotDistanceInfo()
plt.suptitle("No assimilation, $N_e = $ " + str(numParticles))
print ensemble.getObsMatrix()

In [ ]:
def runAndResample(ensemble, timesteps):
    ensemble.step(timesteps)
    notUsed = 0
    dautils.residualSampling(ensemble)
    


T = 1000
numParticles = 100
obs_vars = '100'
ensemble = Lorenz63Ensemble(cl_ctx, numParticles, T=T, obs_vars=obs_vars)
                            #obs_matrix_diag=obsmat)
                            #model_error=0, init_error=0.1)
                            #observation_variance=5)

obs_frequency = 25
for i in range(int(T/obs_frequency)-4):
    runAndResample(ensemble, obs_frequency)
ensemble.step(1000)
ensemble.plotDistanceInfo()
plt.suptitle("Particle Filter with Residual Resampling, $N_e$ = " + str(numParticles))
print ensemble.getObsMatrix()


In [ ]:
def runAndResample(ensemble, timesteps):
    ensemble.step(timesteps)
    notUsed = 0
    dautils.stochasticUniversalSampling(ensemble,)
    


T = 1000
numParticles = 100
obs_vars = '111'
ensemble = Lorenz63Ensemble(cl_ctx, numParticles, T=T, obs_vars=obs_vars)# , init_error=0.0)
                            #obs_matrix_diag=obsmat)
                            #model_error=0, init_error=0.1)
                            #observation_variance=5)

obs_frequency = 25
for i in range(int(T/obs_frequency)-4):
    runAndResample(ensemble, obs_frequency)
ensemble.step(1000)
ensemble.plotDistanceInfo()
plt.suptitle("Particle Filter with Stochastic Universal Resampling, $N_e$ = " + str(numParticles))
print ensemble.getObsMatrix()


In [ ]:
def EnKF(ensemble):
    # Get observations and ensemble state
    obs = ensemble.observeTrueState()
    obs = np.matrix(obs).transpose()
    observed_forecast = ensemble.observeParticles()
    forecast = ensemble.observeParticles(True)
    
    # obtain cov matrices
    P = ensemble.getEnsembleCov()
    R = ensemble.getObservationCovMatrix()
    
    # obtain observation operator (should be particles._observe)
    H = ensemble.getObsMatrix()
    
    # Find the Kalman gain:
    PHT =  np.dot(P, H.transpose())
    HPHT = np.dot(H, PHT)
    invHPHTpR = np.linalg.inv(HPHT + R)
    K = np.dot(PHT, invHPHTpR)
    
    # Find innovation:
    innovation = obs - observed_forecast
    
    analyse_ensemble = np.array(forecast + np.dot(K, innovation))
    ensemble.setParticles(analyse_ensemble)


T = 1000
numParticles = 20
obs_vars = '101'
ensemble = Lorenz63Ensemble(cl_ctx, numParticles, T=T, obs_vars=obs_vars)
                            #obs_matrix_diag=obsmat)
                            #model_error=0, init_error=0.1)
                            #observation_variance=5)

obs_frequency = 25
for i in range(int(T/obs_frequency)-5):
    ensemble.step(obs_frequency)
    EnKF(ensemble)
ensemble.step(1000)
ensemble.plotDistanceInfo()
plt.suptitle("Ensemble Kalman Filter, $N_e$ = " + str(numParticles))

In [ ]:
def SEnKF(ensemble):
    """
    Stochastic Ensemble Kalman Filter
    """
    # Get observations and ensemble state
    obs = ensemble.observeTrueState()
    obs = np.matrix(obs).transpose()
    observed_forecast = ensemble.observeParticles()
    forecast = ensemble.observeParticles(True)
    
    # obtain cov matrices
    P = ensemble.getEnsembleCov()
    R = ensemble.getObservationCovMatrix()
    
    # Perturb the observations of the ensemble:
    for i in range(observed_forecast.shape[1]):
        if ensemble.observation_size == 1:
            observed_forecast[:,i] += np.random.normal(0, R)
        else:
            observed_forecast[:,i] += np.random.multivariate_normal(np.zeros(ensemble.observation_size),
                                                               R)
    
    # obtain observation operator (should be particles._observe)
    H = ensemble.getObsMatrix()
    
    # Find the Kalman gain:
    PHT =  np.dot(P, H.transpose())
    HPHT = np.dot(H, PHT)
    invHPHTpR = np.linalg.inv(HPHT + R)
    K = np.dot(PHT, invHPHTpR)
    
    # Find innovation:
    innovation = obs - observed_forecast
    
    analyse_ensemble = np.array(forecast + np.dot(K, innovation))
    ensemble.setParticles(analyse_ensemble)


T = 1000
numParticles = 30
obs_vars = '001'
ensemble = Lorenz63Ensemble(cl_ctx, numParticles, T=T, obs_vars=obs_vars)
                            #obs_matrix_diag=obsmat)
                            #model_error=0, init_error=0.1)
                            #observation_variance=5)

obs_frequency = 25
for i in range(int(T/obs_frequency)-5):
    ensemble.step(obs_frequency)
    SEnKF(ensemble)
ensemble.step(1000)
ensemble.plotDistanceInfo()
plt.suptitle("Stochastic Ensemble Kalman Filter, $N_e$= " + str(numParticles))


In [ ]:
def ETKF(ensemble):
    """
    Ensemble Transform Kalman Filter
    """
    # Get forecast, forecast mean, and forecast perturbation
    forecast = ensemble.observeParticles(True)
    forecast_mean = np.matrix(np.mean(forecast, axis=1)).T
    forecast_perturbations = forecast - forecast_mean
    #print forecast_perturbations

    Ne = ensemble.getNumParticles()

    # Forecast in observation space
    observed_forecast = ensemble.observeParticles()
    observed_forecast_mean = np.matrix(np.mean(observed_forecast, axis=1)).T
    S = observed_forecast - observed_forecast_mean

    # Observation and innovation
    observation = np.matrix(ensemble.observeTrueState()).T
    innovation = observation - observed_forecast
    R = ensemble.getObservationCovMatrix()
    Rinv = np.linalg.inv(R)

    # Obtain (TT)^-1 and its eigenvalue decomposition
    STRinv = np.dot(S.T, Rinv)
    TTinv = np.eye(Ne) + (1.0/(Ne-1.0))*np.dot(STRinv, S)
    #print TTinv
    TTinv_symm = (TTinv + TTinv.T)/2.0 # since TTinv is supposed to be symmetric by construction
    sigma, U = np.linalg.eigh(TTinv_symm)
    sigmaInv = 1.0/sigma
    sigmaInvRoot = np.sqrt(sigmaInv)

    # Obtain mean w
    UsigmaInvUT = np.dot( np.dot(U, np.diag(sigmaInv)), U.T)
    STRinvd = np.dot(STRinv, innovation)
    w_mean = (1.0/np.sqrt(Ne-1.0))*np.dot(UsigmaInvUT, STRinvd)

    # Obtain w perturbation
    W_perturbation = np.dot( np.dot(U, np.diag(sigmaInvRoot)), U.T)
    #print W_perturbation

    analysis_mean = forecast_mean + np.dot(forecast_perturbations, w_mean)

    analysis_perturbation = np.dot(forecast_perturbations, W_perturbation)

    analysis = np.array(analysis_mean + analysis_perturbation)
    ensemble.setParticles(analysis)

T = 1000
numParticles = 100
obs_vars = '111'
ensemble = Lorenz63Ensemble(cl_ctx, numParticles, T=T, obs_vars=obs_vars)
                            #obs_matrix_diag=obsmat)
                            #model_error=0, init_error=0.1)
                            #observation_variance=5)

dontWork_doNothing = True
if not dontWork_doNothing:
    obs_frequency = 25
    for i in range(int(T/obs_frequency)-5):
        ensemble.step(obs_frequency)
        ETKF(ensemble)
        print "obs " + str(i)
    ensemble.step(1000)
    ensemble.plotDistanceInfo()
    plt.suptitle("Ensemble Transform Kalman Filter")
else:
    print "This don't work - doing nothing"

In [ ]:
def ETKFfromPaper(ensemble):
    """
    Ensemble Transform Kalman Filter
    """
    # Get forecast, forecast mean, and forecast perturbation
    Ne = ensemble.getNumParticles()
    forecast = ensemble.observeParticles(True)
    forecast_mean = np.matrix(np.mean(forecast, axis=1)).T
    forecast_perturbations = forecast - forecast_mean

    observed_forecast = ensemble.observeParticles()
    observed_forecast_mean = np.matrix(np.mean(observed_forecast, axis=1)).T

    # first difference - innovation from forecast mean:
    observation = np.matrix(ensemble.observeTrueState()).T
    d = observation - observed_forecast_mean
    R = ensemble.getObservationCovMatrix()
    #R = np.dot(R, R)
    Rinv = np.linalg.inv(R)

    S = observed_forecast - observed_forecast_mean

    # Basic is settled, now for some different calculations:
    C = np.dot(Rinv, S)
    A1 = (Ne - 1.0)*np.eye(Ne)
    #A1 = np.eye(Ne)
    A2 = A1 + np.dot(S.T, C) #/(Ne - 1.0)
    D = np.dot(C.T, d)
    A3 = (A2 + A2.T)/2.0

    # Eigenvalue decomposition
    sigma, U = np.linalg.eigh(A3)
    w = np.dot(U.T, D)

    for j in range(Ne):
        w[j] = w[j]/sigma[j]

    w_mean = np.dot(U, w) #*(np.sqrt(Ne - 1.0))

    W_pert = np.matrix(np.zeros((Ne, Ne)))
    for j in range(Ne):
        #W_pert[j,:] = (np.sqrt(sigma[j]))*U[j,:] # Don't work, but according to paper
        W_pert[j,:] = (1.0/np.sqrt(sigma[j]))*U[j,:] # gives better result for the mean, but still bad...

    W_pert = np.dot(W_pert, U.T)
    #W = W_pert*np.sqrt(Ne-1.0) + w_mean
    #analysis = np.array(forecast + np.dot(forecast_perturbations, W))
    
    analysis_mean = forecast_mean + np.dot(forecast_perturbations, w_mean)
    analysis_perturbations = np.dot(forecast_perturbations, W_pert)
    analysis = np.array(analysis_mean + analysis_perturbations )
    ensemble.setParticles(analysis)

T = 1000
numParticles = 100
obs_vars = '110'
ensemble = Lorenz63Ensemble(cl_ctx, numParticles, T=T, obs_vars=obs_vars)
                            #obs_matrix_diag=obsmat)
                            #model_error=0, init_error=0.1)
                            #observation_variance=5)

obs_frequency = 25
for i in range(int(T/obs_frequency)-5):
    ensemble.step(obs_frequency)
    ETKFfromPaper(ensemble)
    #print "obs " + str(i)
ensemble.step(1000)
ensemble.plotDistanceInfo()
plt.suptitle("Ensemble Transform Kalman Filter, $N_e$ = " + str(numParticles))


In [ ]:
#Find covariance of ensemble
H = ensemble.getObsMatrix()

mean = ensemble.getEnsembleMean()[:,-1]
print mean
states = ensemble.observeParticles(True)
observedStates = np.dot(H, states)
print states.shape
print observedStates.shape

P_forecast = np.zeros((3,3))
for i in range(states.shape[1]):
    P_forecast += np.outer(states[:,i]-mean, states[:,i]-mean)
P_forecast = P_forecast/(ensemble.getNumParticles() - 1.0)
print P_forecast
fig = plt.figure(figsize=(3,3))
plt.imshow(P_forecast, interpolation="None")
plt.colorbar()

In [ ]:
a = np.matrix([1,1,1,1,1,1])
b = np.matrix([10,10,10,10,0,0])
np.sqrt(np.dot(a, b.T))
np.linalg.norm(a), np.sqrt(6)
print 1/np.sqrt(0.05)

In [ ]:
np.outer(a,a) - b.T
(1.0/(Ne-1.0))
print Ne

In [ ]:
covariance_matrix = np.matrix([[1,0.5,0.25], [0.5,1,0.5], [0.25,0.5,1]])
print covariance_matrix
print 10*covariance_matrix